In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import pandas as pd
import psycopg2
import numpy as np

from IPython.display import display, HTML

In [2]:
con = psycopg2.connect(dbname='mimic', user='sudarshan', host='/var/run/postgresql')

In [4]:
query = """
select subject_id, hadm_id, icustay_id from icustays limit 10
"""
df = pd.read_sql_query(query, con)

df.head()

,subject_id,hadm_id,icustay_id
0,268,110404,280836
1,269,106296,206613
2,270,188028,220345
3,271,173727,249196
4,272,164716,210407


In [5]:
query = """
select subject_id, hadm_id, icustay_id, outtime-intime as icu_los_interval, extract(epoch from outtime-intime) as icu_los from icustays limit 10
"""
df = pd.read_sql_query(query, con)
df.head()

,subject_id,hadm_id,icustay_id,icu_los_interval,icu_los
0,268,110404,280836,3 days 05:58:33,280713.0
1,269,106296,206613,3 days 06:41:28,283288.0
2,270,188028,220345,2 days 21:27:09,250029.0
3,271,173727,249196,2 days 01:26:22,177982.0
4,272,164716,210407,1 days 14:53:09,139989.0


Below LOS is represented as *fractional days*.

In [6]:
query = """
select subject_id, hadm_id, icustay_id, extract(epoch from outtime-intime)/60.0/60.0/24.0 as icu_los from icustays limit 10
"""
df = pd.read_sql_query(query, con)
df.head()

,subject_id,hadm_id,icustay_id,icu_los
0,268,110404,280836,3.248993
1,269,106296,206613,3.278796
2,270,188028,220345,2.893854
3,271,173727,249196,2.059977
4,272,164716,210407,1.620243


In [7]:
query = """
with co as 
(
    select subject_id, hadm_id, icustay_id, extract(epoch from outtime-intime)/60.0/60.0/24.0 as icu_los from icustays limit 10
)
select co.subject_id, co.hadm_id, co.icustay_id, co.icu_los from co where icu_los >= 2
"""
df = pd.read_sql_query(query, con)
df

,subject_id,hadm_id,icustay_id,icu_los
0,268,110404,280836,3.248993
1,269,106296,206613,3.278796
2,270,188028,220345,2.893854
3,271,173727,249196,2.059977
4,274,130546,254851,8.814259
5,275,129886,219649,7.131412


In [8]:
query = """
with co as 
(
    select subject_id, hadm_id, icustay_id, extract(epoch from outtime-intime)/60.0/60.0/24.0 as icu_los from icustays limit 10
)
select co.subject_id, co.hadm_id, co.icustay_id, co.icu_los,
case when co.icu_los < 2 then 1
else 0 end as exlusion_los
from co
"""
df = pd.read_sql_query(query, con)
df

,subject_id,hadm_id,icustay_id,icu_los,exlusion_los
0,268,110404,280836,3.248993,0
1,269,106296,206613,3.278796,0
2,270,188028,220345,2.893854,0
3,271,173727,249196,2.059977,0
4,272,164716,210407,1.620243,1
5,273,158689,241507,1.486181,1
6,274,130546,254851,8.814259,0
7,275,129886,219649,7.131412,0
8,276,135156,206327,1.337836,1
9,277,171601,272866,0.731273,1


In [9]:
query = """
with co as 
(
    select icu.subject_id, icu.hadm_id, icu.icustay_id, extract(epoch from outtime-intime)/60.0/60.0/24.0 as icu_los, icu.intime - pat.dob as age from icustays icu
    inner join patients pat
    on icu.subject_id = pat.subject_id limit 10
)
select co.subject_id, co.hadm_id, co.icustay_id, co.icu_los, co.age,
case when co.icu_los < 2 then 1
else 0 end as exclusion_los
from co
"""
df = pd.read_sql_query(query, con)
df

,subject_id,hadm_id,icustay_id,icu_los,age,exclusion_los
0,2,163353,243653,0.091829,0 days 21:20:07,1
1,3,145834,211552,6.064560,27950 days 19:10:11,0
2,4,185777,294638,1.678472,17475 days 00:29:31,1
3,5,178980,214757,0.084444,0 days 06:04:24,1
4,6,107064,228232,3.672917,24084 days 21:30:54,0
5,7,118037,278444,0.267720,0 days 15:35:29,1
6,7,118037,236754,0.739097,2 days 03:26:01,1
7,8,159514,262299,1.075521,0 days 12:36:10,1
8,9,150750,220597,5.323056,15263 days 13:07:02,0
9,10,184167,288409,8.092106,0 days 11:39:05,0


In [10]:
query = """
with co as 
(
    select icu.subject_id, icu.hadm_id, icu.icustay_id, extract(epoch from outtime-intime)/60.0/60.0/24.0 as icu_los, icu.intime - pat.dob as age from icustays icu
    inner join patients pat
    on icu.subject_id = pat.subject_id limit 10
)
select co.subject_id, co.hadm_id, co.icustay_id, co.icu_los, co.age,
extract('year' from co.age) as age_extract_year,
extract('year' from co.age) + extract('months' from co.age)/12.0 + extract('days' from co.age)/365.242 + extract('hours' from co.age)/24.0/365.242 as age_extract_precise,
extract('epoch' from co.age)/60.0/60.0/24.0/365.242 as age_extract_epoch,
case when co.icu_los < 2 then 1
else 0 end as exclusion_los
from co
"""
df = pd.read_sql_query(query, con)
df

,subject_id,hadm_id,icustay_id,icu_los,age,age_extract_year,age_extract_precise,age_extract_epoch,exclusion_los
0,2,163353,243653,0.091829,0 days 21:20:07,0.0,0.002396,0.002434,1
1,3,145834,211552,6.064560,27950 days 19:10:11,0.0,76.526773,76.526792,0
2,4,185777,294638,1.678472,17475 days 00:29:31,0.0,47.844990,47.845047,1
3,5,178980,214757,0.084444,0 days 06:04:24,0.0,0.000684,0.000693,1
4,6,107064,228232,3.672917,24084 days 21:30:54,0.0,65.942238,65.942297,0
5,7,118037,278444,0.267720,0 days 15:35:29,0.0,0.001711,0.001779,1
6,7,118037,236754,0.739097,2 days 03:26:01,0.0,0.005818,0.005868,1
7,8,159514,262299,1.075521,0 days 12:36:10,0.0,0.001369,0.001438,1
8,9,150750,220597,5.323056,15263 days 13:07:02,0.0,41.790215,41.790228,0
9,10,184167,288409,8.092106,0 days 11:39:05,0.0,0.001255,0.001329,0


In [11]:
query = """
with co as 
(
    select icu.subject_id, icu.hadm_id, icu.icustay_id, extract(epoch from outtime-intime)/60.0/60.0/24.0 as icu_los, extract(epoch from icu.intime - pat.dob)/60.0/60.0/24.0/365.242 as age from icustays icu
    inner join patients pat
    on icu.subject_id = pat.subject_id limit 10
)
select co.subject_id, co.hadm_id, co.icustay_id, co.icu_los, co.age,
case when co.icu_los < 2 then 1
else 0 end as exclusion_los,
case when co.age < 16 then 1
else 0 end as exclusion_age
from co
"""
df = pd.read_sql_query(query, con)
df

,subject_id,hadm_id,icustay_id,icu_los,age,exclusion_los,exclusion_age
0,2,163353,243653,0.091829,0.002434,1,1
1,3,145834,211552,6.064560,76.526792,0,0
2,4,185777,294638,1.678472,47.845047,1,0
3,5,178980,214757,0.084444,0.000693,1,1
4,6,107064,228232,3.672917,65.942297,0,0
5,7,118037,278444,0.267720,0.001779,1,1
6,7,118037,236754,0.739097,0.005868,1,1
7,8,159514,262299,1.075521,0.001438,1,1
8,9,150750,220597,5.323056,41.790228,0,0
9,10,184167,288409,8.092106,0.001329,0,1


In [12]:
query = """
with co as 
(
    select icu.subject_id, icu.hadm_id, icu.icustay_id,
    extract(epoch from outtime-intime)/60.0/60.0/24.0 as icu_los,
    extract(epoch from icu.intime - pat.dob)/60.0/60.0/24.0/365.242 as age,
    rank() over (partition by icu.subject_id order by icu.intime) as icustay_id_order
    from icustays icu inner join patients pat
    on icu.subject_id = pat.subject_id limit 10
)
select co.subject_id, co.hadm_id, co.icustay_id, co.icu_los, co.age, co.icustay_id_order,
case when co.icu_los < 2 then 1
else 0 end as exclusion_los,
case when co.age < 16 then 1
else 0 end as exclusion_age,
case when co.icustay_id_order != 1 then 1
else 0 end as exlusion_first_stay
from co
"""
df = pd.read_sql_query(query, con)
df

,subject_id,hadm_id,icustay_id,icu_los,age,icustay_id_order,exclusion_los,exclusion_age,exlusion_first_stay
0,2,163353,243653,0.091829,0.002434,1,1,1,0
1,3,145834,211552,6.064560,76.526792,1,0,0,0
2,4,185777,294638,1.678472,47.845047,1,1,0,0
3,5,178980,214757,0.084444,0.000693,1,1,1,0
4,6,107064,228232,3.672917,65.942297,1,0,0,0
5,7,118037,278444,0.267720,0.001779,1,1,1,0
6,7,118037,236754,0.739097,0.005868,2,1,1,1
7,8,159514,262299,1.075521,0.001438,1,1,1,0
8,9,150750,220597,5.323056,41.790228,1,0,0,0
9,10,184167,288409,8.092106,0.001329,1,0,1,0


In [13]:
query = """
select hadm_id, curr_service,
case when curr_service like '%SURG' then 1
when curr_service = 'ORTHO' then 1
else 0 end as surgical
from services limit 10
"""
df = pd.read_sql_query(query, con)
df

,hadm_id,curr_service,surgical
0,135879,MED,0
1,135879,TSURG,1
2,173064,CMED,0
3,129194,NB,0
4,194246,NB,0
5,146746,NBB,0
6,139351,NB,0
7,161042,NB,0
8,191025,MED,0
9,137370,NB,0


In [14]:
query = """
select icu.hadm_id, icu.icustay_id, se.curr_service,
case when curr_service like '%SURG' then 1
when curr_service = 'ORTHO' then 1
else 0 end as surgical
from icustays icu
left join services se 
on icu.hadm_id = se.hadm_id
limit 10
"""
df = pd.read_sql_query(query, con)
df

,hadm_id,icustay_id,curr_service,surgical
0,100001,275225,MED,0
1,100003,209281,MED,0
2,100006,291788,MED,0
3,100006,291788,OMED,0
4,100007,217937,SURG,1
5,100009,253656,CSURG,1
6,100010,271147,GU,0
7,100011,214619,TRAUM,0
8,100012,239289,SURG,1
9,100016,217590,MED,0


In [15]:
query = """
select icu.hadm_id, icu.icustay_id, se.curr_service,
case when curr_service like '%SURG' then 1
when curr_service = 'ORTHO' then 1
else 0 end as surgical
from icustays icu
left join services se 
on icu.hadm_id = se.hadm_id
and se.transfertime < icu.intime + interval '12' hour
limit 10
"""
df = pd.read_sql_query(query, con)
df

,hadm_id,icustay_id,curr_service,surgical
0,100001,275225,MED,0
1,100003,209281,MED,0
2,100006,291788,MED,0
3,100007,217937,SURG,1
4,100009,253656,CSURG,1
5,100010,271147,GU,0
6,100011,214619,TRAUM,0
7,100012,239289,SURG,1
8,100016,217590,MED,0
9,100017,258320,MED,0


In [16]:
query = """
with serv as (
select icu.hadm_id, icu.icustay_id, se.curr_service,
case when curr_service like '%SURG' then 1
when curr_service = 'ORTHO' then 1
else 0 end as surgical,
rank() over (partition by icu.hadm_id order by se.transfertime desc) as rank
from icustays icu left join services se
on icu.hadm_id = se.hadm_id
and se.transfertime < icu.intime + interval '12' hour
limit 10
)
select hadm_id, icustay_id, curr_service, surgical from serv where rank = 1
"""
df = pd.read_sql_query(query, con)
df

,hadm_id,icustay_id,curr_service,surgical
0,100001,275225,MED,0
1,100003,209281,MED,0
2,100006,291788,MED,0
3,100007,217937,SURG,1
4,100009,253656,CSURG,1
5,100010,271147,GU,0
6,100011,214619,TRAUM,0
7,100012,239289,SURG,1
8,100016,217590,MED,0
9,100017,258320,MED,0


In [17]:
query = """
with co as 
(
    select icu.subject_id, icu.hadm_id, icu.icustay_id,
    extract(epoch from outtime-intime)/60.0/60.0/24.0 as icu_los,
    extract(epoch from icu.intime - pat.dob)/60.0/60.0/24.0/365.242 as age,
    rank() over (partition by icu.subject_id order by icu.intime) as icustay_id_order
    from icustays icu inner join patients pat
    on icu.subject_id = pat.subject_id
),
serv as (
select icu.hadm_id, icu.icustay_id, se.curr_service,
case when curr_service like '%SURG' then 1
when curr_service = 'ORTHO' then 1
else 0 end as surgical,
rank() over (partition by icu.hadm_id order by se.transfertime desc) as rank
from icustays icu left join services se
on icu.hadm_id = se.hadm_id
and se.transfertime < icu.intime + interval '12' hour
)
select co.subject_id, co.hadm_id, co.icustay_id, co.icu_los, co.age, co.icustay_id_order,
case when co.icu_los < 2 then 1
else 0 end as exclusion_los,
case when co.age < 16 then 1
else 0 end as exclusion_age,
case when co.icustay_id_order != 1 then 1
else 0 end as exlusion_first_stay,
case when serv.surgical = 1 then 1
else 0 end as exclusion_surgical
from co
left join serv on co.icustay_id = serv.icustay_id and serv.rank = 1
"""
df = pd.read_sql_query(query, con)
df.head(10)

,subject_id,hadm_id,icustay_id,icu_los,age,icustay_id_order,exclusion_los,exclusion_age,exlusion_first_stay,exclusion_surgical
0,2,163353,243653,0.091829,0.002434,1,1,1,0,0
1,3,145834,211552,6.064560,76.526792,1,0,0,0,1
2,4,185777,294638,1.678472,47.845047,1,1,0,0,0
3,5,178980,214757,0.084444,0.000693,1,1,1,0,0
4,6,107064,228232,3.672917,65.942297,1,0,0,0,1
5,7,118037,278444,0.267720,0.001779,1,1,1,0,0
6,7,118037,236754,0.739097,0.005868,2,1,1,1,0
7,8,159514,262299,1.075521,0.001438,1,1,1,0,0
8,9,150750,220597,5.323056,41.790228,1,0,0,0,0
9,10,184167,288409,8.092106,0.001329,1,0,1,0,0


In [18]:
print('{:20s} {:5d}'.format('Observations', df.shape[0]))
idxExcl = np.zeros(df.shape[0],dtype=bool)
for col in df.columns:
    if "exclusion_" in col:
        print('{:20s} {:5d} ({:2.2f}%)'.format(col, df[col].sum(), df[col].sum()*100.0/df.shape[0]))
        idxExcl = (idxExcl) | (df[col]==1)

# print a summary of how many were excluded in total
print('')
print('{:20s} {:5d} ({:2.2f}%)'.format('Total excluded', np.sum(idxExcl), np.sum(idxExcl)*100.0/df.shape[0]))

Observations         61534
exclusion_los        29213 (47.47%)
exclusion_age         8109 (13.18%)
exclusion_surgical   18225 (29.62%)

Total excluded       42945 (69.79%)


In [19]:
con.close()